In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, lit, expr
from pyspark.sql.types import StructType, StringType, LongType, DoubleType
from pyspark.sql import SparkSession

In [3]:
# Define schema for incoming Kafka JSON value
schema = StructType() \
    .add("asset_id", StringType()) \
    .add("timestamp", LongType()) \
    .add("status", DoubleType())  # status: 0, 1, 2, 408


In [4]:
packages = ["org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3"]

spark = SparkSession.builder \
    .appName("MachineDataAggregator") \
    .master("local[1]") \
    .config("spark.ui.enabled", "true") \
    .config("spark.ui.port", "4040") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.driver.memory", "1G") \
    .config("spark.sql.shuffle.partitions", 2) \
    .config("spark.jars.packages", ",".join(packages)) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")


your 131072x1 screen size is bogus. expect trouble
25/05/31 13:29:25 WARN Utils: Your hostname, DESKTOP-KKQCLCE resolves to a loopback address: 127.0.1.1; using 172.24.89.73 instead (on interface eth0)
25/05/31 13:29:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kr/.ivy2/cache
The jars for the packages stored in: /home/kr/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8955049f-90b3-47b6-abf7-f55d7157a8c8;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.0 in local-m2-cache
	found org.apache.commons#commons-pool2;2.11.1 in local-m2-cache
:: resolution report :: resolve 2967ms :: artifac

In [5]:
# Read from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "machine_data") \
    .option("startingOffsets", "latest") \
    .load()


In [6]:
    
# Parse value as JSON
json_df = df.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*") \
    .withColumn("event_time", (col("timestamp") / 1000).cast("timestamp"))



In [ ]:
-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|7       |1          |0           |0              |0              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|8       |0          |0           |0              |1              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|1       |0          |0           |1              |0              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|2       |0          |0           |0              |1              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|3       |0          |0           |0              |1              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|4       |0          |0           |0              |1              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|5       |0          |1           |0              |0              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|6       |0          |0           |0              |1              |1        |
|2025-05-31 13:31:20.456|2025-05-31 13:31:00|9       |0          |0           |0              |1              |1        |
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+


In [22]:


# Define 1-minute window for aggregation
windowed_df = json_df \
    .withWatermark("event_time", "1 minutes") \
    .groupBy(
        window(col("event_time"), "1 minute"),
        col("asset_id")
    ).agg(
        expr("sum(case when status = 1 then 5 else 0 end)").alias("run_seconds"),
        expr("sum(case when status = 2 then 5 else 0 end)").alias("idle_seconds"),
        expr("sum(case when status = 0 then 5 else 0 end)").alias("stopped_seconds"),
        expr("sum(case when status = 408 then 5 else 0 end)").alias("offline_seconds")
    ) \
    .withColumn("total_seconds", expr("run_seconds + idle_seconds + stopped_seconds + offline_seconds")) \
    .withColumn("data_loss", expr("GREATEST(60 - total_seconds, 0)")) \
    .withColumn("minute", col("window.start")) \
    .withColumn("ts", expr("CAST(current_timestamp() AS timestamp)")) \
    .select(
        "ts",
        "minute",
        "asset_id",
        "run_seconds",
        "idle_seconds",
        "stopped_seconds",
        "offline_seconds",
        "data_loss"
    )

In [23]:
query = windowed_df \
    .selectExpr("CAST(asset_id AS STRING) AS key", "to_json(struct(*)) AS value") \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "machine_1_minute") \
    .option("checkpointLocation", "/tmp/spark-checkpoint/machine_1_minute") \
    .outputMode("update") \
    .queryName("aggregates_to_kafka") \
    .start()


25/05/31 13:44:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/05/31 13:44:19 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/31 13:57:02 WARN Sender: [Producer clientId=producer-1] Got error produce response with correlation id 560 on topic-partition machine_1_minute-2, retrying (2147483646 attempts left). Error: NOT_LEADER_OR_FOLLOWER
25/05/31 13:57:02 WARN Sender: [Producer clientId=producer-1] Received invalid metadata error in produce request on partition machine_1_minute-2 due to org.apache.kafka.common.errors.NotLeaderOrFollowerException: For requests intended only for the leader, this error indicates that the broker is not the current leader. For requests intended for any replica, this error indicates that the broker is not a replica of the topic partition.. Going to request metadata update now
25/05/31 13:57:02 WARN Sender: [Producer clientId=producer-1] Got error produce response with correlation id 

In [ ]:

# Output to console (or replace with write to file/db/sink)
# query = windowed_df.writeStream \
#     .outputMode("update") \
#     .format("console") \
#     .option("truncate", "false") \
#     .option("numRows", 10) \
#     .queryName("aggregates") \
#     .start()    

#query.awaitTermination()

25/05/31 13:36:03 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-06ae2da5-2722-4bd6-85f8-8ec633e5881e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/31 13:36:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/05/31 13:36:03 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+------+--------+-----------+------------+---------------+---------------+---------+
|ts |minute|asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+---+------+--------+-----------+------------+---------------+---------------+---------+
+---+------+--------+-----------+------------+---------------+---------------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:36:05.953|2025-05-31 13:36:00|2       |5          |0           |0              |0              |1        |
|2025-05-31 13:36:05.953|2025-05-31 13:36:00|3       |5          |0           |0              |0              |1        |
|2025-05-31 13:36:05.953|2025-05-31 13:36:00|4       |0          |0           |5              |0              |1        |
|2025-05-31 13:36:05.953|2025-05-31 13:36:00|5       |0          |0           |0              |5              |1        |
|2025-05-31 13:36:05.953|2025-05-31 13:36:00|6   

-------------------------------------------
Batch: 2
-------------------------------------------
+---+------+--------+-----------+------------+---------------+---------------+---------+
|ts |minute|asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+---+------+--------+-----------+------------+---------------+---------------+---------+
+---+------+--------+-----------+------------+---------------+---------------+---------+

-------------------------------------------
Batch: 3
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:36:10.882|2025-05-31 13:36:00|2       |5          |5           |0   

In [18]:
query.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

-------------------------------------------
Batch: 4
-------------------------------------------
+---+------+--------+-----------+------------+---------------+---------------+---------+
|ts |minute|asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+---+------+--------+-----------+------------+---------------+---------------+---------+
+---+------+--------+-----------+------------+---------------+---------------+---------+



-------------------------------------------
Batch: 5
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:36:15.612|2025-05-31 13:36:00|2       |5          |10          |0              |0              |1        |
|2025-05-31 13:36:15.612|2025-05-31 13:36:00|3       |5          |0           |5              |5              |1        |
|2025-05-31 13:36:15.612|2025-05-31 13:36:00|4       |5          |5           |5              |0              |1        |
|2025-05-31 13:36:15.612|2025-05-31 13:36:00|5       |0          |0           |0              |15             |1        |
|2025-05-31 13:36:15.612|2025-05-31 13:36:00|6   

-------------------------------------------
Batch: 7
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:36:22.004|2025-05-31 13:36:00|2       |5          |10          |5              |0              |1        |
|2025-05-31 13:36:22.004|2025-05-31 13:36:00|3       |10         |0           |5              |5              |1        |
|2025-05-31 13:36:22.004|2025-05-31 13:36:00|4       |5          |5           |10             |0              |1        |
|2025-05-31 13:36:22.004|2025-05-31 13:36:00|5       |0          |0           |5              |15             |1        |
|2025-05-31 13:36:22.004|2025-05-31 13:36:00|6   

-------------------------------------------
Batch: 8
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:36:29.762|2025-05-31 13:36:00|2       |10         |10          |5              |0              |1        |
|2025-05-31 13:36:29.762|2025-05-31 13:36:00|3       |15         |0           |5              |5              |1        |
|2025-05-31 13:36:29.762|2025-05-31 13:36:00|4       |5          |5           |15             |0              |1        |
|2025-05-31 13:36:29.762|2025-05-31 13:36:00|5       |0          |5           |5              |15             |1        |
|2025-05-31 13:36:29.762|2025-05-31 13:36:00|6   

...

-------------------------------------------
Batch: 43
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:38:12.125|2025-05-31 13:38:00|3       |5          |0           |5              |5              |1        |
|2025-05-31 13:38:12.125|2025-05-31 13:38:00|4       |10         |5           |0              |0              |1        |
|2025-05-31 13:38:12.125|2025-05-31 13:38:00|5       |5          |0           |10             |0              |1        |
|2025-05-31 13:38:12.125|2025-05-31 13:38:00|7       |0          |10          |5              |0              |1        |
|2025-05-31 13:38:12.125|2025-05-31 13:38:00|9  

-------------------------------------------
Batch: 45
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:38:17.114|2025-05-31 13:38:00|3       |5          |0           |5              |10             |1        |
|2025-05-31 13:38:17.114|2025-05-31 13:38:00|4       |10         |10          |0              |0              |1        |
|2025-05-31 13:38:17.114|2025-05-31 13:38:00|5       |5          |5           |10             |0              |1        |
|2025-05-31 13:38:17.114|2025-05-31 13:38:00|7       |0          |10          |10             |0              |1        |
|2025-05-31 13:38:17.114|2025-05-31 13:38:00|9  

-------------------------------------------
Batch: 46
-------------------------------------------
+---+------+--------+-----------+------------+---------------+---------------+---------+
|ts |minute|asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+---+------+--------+-----------+------------+---------------+---------------+---------+
+---+------+--------+-----------+------------+---------------+---------------+---------+



-------------------------------------------
Batch: 47
-------------------------------------------
+----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                    |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:38:22.14|2025-05-31 13:38:00|3       |5          |0           |10             |10             |1        |
|2025-05-31 13:38:22.14|2025-05-31 13:38:00|4       |10         |15          |0              |0              |1        |
|2025-05-31 13:38:22.14|2025-05-31 13:38:00|5       |5          |10          |10             |0              |1        |
|2025-05-31 13:38:22.14|2025-05-31 13:38:00|7       |5          |10          |10             |0              |1        |
|2025-05-31 13:38:22.14|2025-05-31 13:38:00|9       |5 

-------------------------------------------
Batch: 48
-------------------------------------------
+---+------+--------+-----------+------------+---------------+---------------+---------+
|ts |minute|asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+---+------+--------+-----------+------------+---------------+---------------+---------+
+---+------+--------+-----------+------------+---------------+---------------+---------+

-------------------------------------------
Batch: 49
-------------------------------------------
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|ts                     |minute             |asset_id|run_seconds|idle_seconds|stopped_seconds|offline_seconds|data_loss|
+-----------------------+-------------------+--------+-----------+------------+---------------+---------------+---------+
|2025-05-31 13:38:31.889|2025-05-31 13:38:00|3       |5          |0           |15

In [ ]:
query.stop()